In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**STEP 1: READ AN AUDIO FILE AND TRANSLATE TO ENGLISH TEXT**

In [ ]:
audio_file_wav = '/content/drive/Shareddrives/298A/Code_directory/sample audio inputs/a-fireman-arrived-at-the-car-accident.wav'

In [ ]:
!pip install --upgrade google-cloud-speech

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.2/275.2 kB 5.2 MB/s eta 0:00:00


In [ ]:
import os
from google.cloud import speech

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/drive/Shareddrives/298A/Code_directory/speech-to-text-382212-46fc5777a765.json'
speech_client = speech.SpeechClient()

with open(audio_file_wav, 'rb') as f1:
    byte_data = f1.read()
audio_wav = speech.RecognitionAudio(content = byte_data)

config_wav = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    audio_channel_count=2,
    enable_automatic_punctuation=True,
    language_code = 'en-US'
)

response_standard_wav = speech_client.recognize(
    config=config_wav,
    audio=audio_wav
    )

In [ ]:
audio_to_text = response_standard_wav.results[0].alternatives[0].transcript

In [ ]:
print(audio_to_text)

A fireman arrived at a car accident.


**STEP 2: CONVERT TEXT SEQUENCE TO GLOSS SEQUENCE**

In [ ]:
# Install the most re version of TensorFlow to use the improved
# masking support for `tf.keras.layers.MultiHeadAttention`.
# !apt install --allow-change-held-packages libcudnn8=8.6.0.163-1+cuda11.8  # Need to check about this library as it is not available for mac system
!pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
!pip install protobuf~=3.20.3
!pip install -q tensorflow_datasets
!pip install -q -U tensorflow-text tensorflow
!pip install transformers --quiet
!pip install ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 53.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6082 sha256=605d4d4761dad00f259ddb75322fb06dae104744fdc91625aa0a1aae81cf9509
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg


In [ ]:
import logging
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow as tf
import tensorflow_text
from tensorflow.data import AUTOTUNE
import collections
import os
import pathlib
import re
import string
import sys
import tempfile

In [ ]:
%cd /content/drive/Shareddrives/298A/Transformer_Inference

/content/drive/Shareddrives/298A/Transformer_Inference


In [ ]:
# tokenizer = tf.saved_model.load('tx_gl_converter')
# tokenizer.tx.get_vocab_size().numpy()

In [ ]:
tx_gl_translation = tf.saved_model.load('tx-gl-translator')

In [ ]:
gloss_text = tx_gl_translation(audio_to_text).numpy()

In [ ]:
converted_sentence = gloss_text.decode('utf-8')  # Convert bytes to string
converted_sentence = converted_sentence.replace(' .', '')  # Convert bytes to string
converted_sentence = converted_sentence.replace('x', '')  # Replace 'x' with 'you'
converted_sentence = converted_sentence.replace('desc', '')  # Replace 'desc' with 'you'
converted_sentence = converted_sentence.replace('  ', '')  # Replace double space with single space

gloss_text = converted_sentence

In [ ]:
print(gloss_text)

fireman arrive at car accident


**STEP 3: GLOSS TO POSEIDS**

In [ ]:
import pandas as pd

meta_df = pd.read_csv("/content/drive/Shareddrives/298A/video-metadata.csv")
liz_meta = meta_df[(meta_df["Consultant"]=="Liz") & ((meta_df["Session"]=="ASL_2008_01_11") | (meta_df["Session"]=="ASL_2008_01_18"))]

In [ ]:
liz_meta.head()

,Gloss Variant,Consultant,Session,Scene,Start,End,id,session_scene,session_scene_id,is_corrupt
2,#AC,Liz,ASL_2008_01_11,4,3285,3325,2,ASL_2008_01_11-4,76,0
7,#ALL,Liz,ASL_2008_01_11,12,275,300,7,ASL_2008_01_11-12,47,0
13,#ASL,Liz,ASL_2008_01_11,23,680,705,13,ASL_2008_01_11-23,58,0
18,#BACK,Liz,ASL_2008_01_11,28,1136,1171,18,ASL_2008_01_11-28,63,0
24,#BUS,Liz,ASL_2008_01_11,59,1370,1418,24,ASL_2008_01_11-59,97,0


In [ ]:
import re

# Function to remove special characters from suffix and prefix
def remove_special_characters(text):
    # Use regular expression to remove special characters from suffix and prefix
    cleaned_text = re.sub(r'^[^a-zA-Z0-9]+|[^a-zA-Z0-9]+$', '', text)
    return cleaned_text

# Apply the function to the 'text' column
liz_meta['cleaned_gloss_variant'] = liz_meta['Gloss Variant'].apply(remove_special_characters)

<ipython-input-16-313f5a04300b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  liz_meta['cleaned_gloss_variant'] = liz_meta['Gloss Variant'].apply(remove_special_characters)


In [ ]:
gloss_sequence = gloss_text

# Split the sentence into words
gloss_list = gloss_sequence.split()

# Print the list of words
print(gloss_list)

['fireman', 'arrive', 'at', 'car', 'accident']


In [ ]:
pose_ids = list()
for term in gloss_list:
  ids = liz_meta[liz_meta["cleaned_gloss_variant"] == term.upper()]["id"].values
  if len(ids) > 0:
    pose_ids.append(ids[0])

In [ ]:
pose_ids

[4100, 1165, 2057, 836]

**STEP 4: EXTRACTING FRAMES FOR GLOSSES VIDEO CLIPS AND GENERATING SKELETONS USING DWPOSE DETECTORS**

In [ ]:
%cd /content

/content


In [ ]:
!git clone https://github.com/patrickvonplaten/controlnet_aux.git

Cloning into 'controlnet_aux'...
remote: Enumerating objects: 1341, done.
remote: Counting objects: 100% (248/248), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 1341 (delta 140), reused 139 (delta 115), pack-reused 1093
Receiving objects: 100% (1341/1341), 1.21 MiB | 7.75 MiB/s, done.
Resolving deltas: 100% (521/521), done.


In [ ]:
!pip install -q controlnet-aux==0.0.7
!pip install -U openmim
!mim install mmengine
!mim install "mmcv>=2.0.1"
!mim install "mmdet>=3.1.0"
!mim install "mmpose>=1.1.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.4/202.4 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 23.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.1/953.1 kB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.1.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.3/450.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 24.4 MB/s eta 0:00:00
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.1.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 MB 4.1 MB/s eta 0:00:00
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.1.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.6 MB/s eta 0:00:00
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.1.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.0/436.0 kB 32.4 MB/s eta 0:00:00
  Created wheel for chumpy: filename=chumpy-0.

In [ ]:
import requests
from PIL import Image
from io import BytesIO

from controlnet_aux.processor import Processor
from controlnet_aux import HEDdetector, MidasDetector, MLSDdetector, OpenposeDetector, PidiNetDetector, NormalBaeDetector, LineartDetector, LineartAnimeDetector, CannyDetector, ContentShuffleDetector, ZoeDetector, MediapipeFaceDetector, SamDetector, LeresDetector, DWposeDetector

# specify configs, ckpts and device, or it will be downloaded automatically and use cpu by default
det_config= '/content/controlnet_aux/src/controlnet_aux/dwpose/yolox_config/yolox_l_8xb8-300e_coco.py'
det_ckpt= 'https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_l_8x8_300e_coco/yolox_l_8x8_300e_coco_20211126_140236-d3bd2b23.pth'
pose_config= '/content/controlnet_aux/src/controlnet_aux/dwpose/dwpose_config/dwpose-l_384x288.py'
pose_ckpt= 'https://huggingface.co/wanghaofan/dw-ll_ucoco_384/resolve/main/dw-ll_ucoco_384.pth'

/usr/local/lib/python3.10/dist-packages/controlnet_aux/mediapipe_face/mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_5m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/usr/local/lib/python3.10/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_11m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_11m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dwpose = DWposeDetector(det_config=det_config, det_ckpt=det_ckpt, pose_config=pose_config, pose_ckpt=pose_ckpt, device=device)

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_l_8x8_300e_coco/yolox_l_8x8_300e_coco_20211126_140236-d3bd2b23.pth


Downloading: "https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_l_8x8_300e_coco/yolox_l_8x8_300e_coco_20211126_140236-d3bd2b23.pth" to /root/.cache/torch/hub/checkpoints/yolox_l_8x8_300e_coco_20211126_140236-d3bd2b23.pth


Loads checkpoint by http backend from path: https://huggingface.co/wanghaofan/dw-ll_ucoco_384/resolve/main/dw-ll_ucoco_384.pth


Downloading: "https://huggingface.co/wanghaofan/dw-ll_ucoco_384/resolve/main/dw-ll_ucoco_384.pth" to /root/.cache/torch/hub/checkpoints/dw-ll_ucoco_384.pth


In [ ]:
import re
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    return sorted(data, key=alphanum_key)

Save continuous frames and running skeleton extraction

In [ ]:
import os

image_no = 0

for id in pose_ids:
  # Filter from Metadata of a single gloss
  gloss_info = liz_meta[liz_meta['id']==id]
  # Variable declaration
  gloss_word = str(gloss_info['cleaned_gloss_variant'].iloc[0])
  print(gloss_word)
  session_id= str(gloss_info['Session'].iloc[0]) #"ASL_2008_01_11"
  scene_id = str(gloss_info['Scene'].iloc[0]) #"scene2-camera1"
  start_frame = int(gloss_info['Start'].iloc[0])
  end_frame = int(gloss_info['End'].iloc[0])
  no_of_frames = (end_frame-start_frame)+1
  url = f"http://csr.bu.edu/asl/asllvd/asl-data2/quicktime/{session_id}/scene{str(scene_id)}-camera1.mov"
  # Create a directory for a gloss word
  !mkdir $gloss_word
  # Command to fetch frames
  !ffmpeg -i $url -vf "select=between(n\,$start_frame\,$end_frame)" -vframes $no_of_frames /content/$gloss_word/frame_%d.jpg
  # Read from the gloss folder and generate skeleton image frames for that gloss
  folder_path = f'/content/{gloss_word}'
  image_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg'))]
  image_outcome = sorted_alphanumeric(image_files)
  print(image_outcome, type(image_outcome))
  for image_file in image_outcome:
    frame_path = os.path.join(folder_path, image_file)
    dest_frame_path = f'/content/drive/Shareddrives/298A/frames/test_A/skeleton_'+str(image_no)+'.jpg'
    print(dest_frame_path)
    img = Image.open(frame_path).convert("RGB").resize((640, 480))
    processed_image_dwpose = dwpose(img, hand_and_face=True, device=device).resize((640, 480))
    processed_image_dwpose.save(dest_frame_path)
    image_no+=1

FIREMAN
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvp

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


/content/drive/Shareddrives/298A/frames/test_A/skeleton_1.jpg
/content/drive/Shareddrives/298A/frames/test_A/skeleton_2.jpg
/content/drive/Shareddrives/298A/frames/test_A/skeleton_3.jpg
/content/drive/Shareddrives/298A/frames/test_A/skeleton_4.jpg
/content/drive/Shareddrives/298A/frames/test_A/skeleton_5.jpg
/content/drive/Shareddrives/298A/frames/test_A/skeleton_6.jpg
/content/drive/Shareddrives/298A/frames/test_A/skeleton_7.jpg
/content/drive/Shareddrives/298A/frames/test_A/skeleton_8.jpg
/content/drive/Shareddrives/298A/frames/test_A/skeleton_9.jpg
/content/drive/Shareddrives/298A/frames/test_A/skeleton_10.jpg
/content/drive/Shareddrives/298A/frames/test_A/skeleton_11.jpg
/content/drive/Shareddrives/298A/frames/test_A/skeleton_12.jpg
/content/drive/Shareddrives/298A/frames/test_A/skeleton_13.jpg
/content/drive/Shareddrives/298A/frames/test_A/skeleton_14.jpg
/content/drive/Shareddrives/298A/frames/test_A/skeleton_15.jpg
/content/drive/Shareddrives/298A/frames/test_A/skeleton_16.jpg
/

In [ ]:
import os

skeleton_path = '/content/drive/Shareddrives/298A/frames/test_A'
no_of_sk = len(os.listdir(skeleton_path))
no_of_sk

90

**STEP 5: GAN INFERENCE OF ALL THE SKELETON FRAMES GENERATED**

In [ ]:
%cd /content/drive/Shareddrives/298A/GAN_Inference/pix2pixHD

/content/drive/Shareddrives/298A/GAN_Inference/pix2pixHD


In [ ]:
!pip install -q dominate tensorflow

In [ ]:
# Conduct inference on test images. Inference results will be stored in "results" directory
!python3 test.py --name pose2sign --dataroot '/content/drive/Shareddrives/298A/frames' --label_nc 0 --no_instance --resize_or_crop None --no_flip --how_many 1000

Traceback (most recent call last):
  File "/content/drive/Shareddrives/298A/GAN_Inference/pix2pixHD/test.py", line 12, in <module>
    opt = TestOptions().parse(save=False)
  File "/content/drive/Shareddrives/298A/GAN_Inference/pix2pixHD/options/base_options.py", line 80, in parse
    torch.cuda.set_device(self.opt.gpu_ids[0])
  File "/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py", line 404, in set_device
    torch._C._cuda_setDevice(device)
  File "/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py", line 298, in _lazy_init
    torch._C._cuda_init()
RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx


In [ ]:
inference_path = '/content/drive/Shareddrives/298A/GAN_Inference/pix2pixHD/results/pose2sign/test_latest/images'
no_of_inferred = len(os.listdir(inference_path))
no_of_inferred

322

To sort the files in alphanumeric order

In [ ]:
import os
from os.path import isfile, join

pathIn = '/content/drive/Shareddrives/298A/GAN_Inference/pix2pixHD/results/pose2sign/test_latest/images'
pathOut = 'new_video_30fps.mp4'
fps = 30
frame_array = []
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]

#for sorting the file names properly
files.sort(key = lambda x: x[5:-4])
files.sort()

In [ ]:
sorted_list = sorted_alphanumeric(files)

Creating the video from inferred frames

In [ ]:
import cv2
import fnmatch

for file in sorted_list:
  if fnmatch.fnmatch(file, '*_synthesized_image.jpg'):
    filename = pathIn +'/'+ file
    #reading each files
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)

    #inserting the frames into an image array
    frame_array.append(img)
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'MP4V'), fps, size)
for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()

Compressing the video

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os

# Input video path
save_path = "/content/drive/Shareddrives/298A/GAN_Inference/pix2pixHD/new_video_30fps.mp4"

# Compressed video path
compressed_path = "/content/drive/Shareddrives/298A/GAN_Inference/pix2pixHD/compressed_generated_30fps_run1.mp4"

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

256

**Generated Sign Language Video**

In [ ]:
# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=700 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)